In [52]:
from sklearn import tree
import pandas as pd
import os
import numpy as np

In [53]:
file = 'clt_temps.csv'


In [54]:
df = pd.read_csv(file, encoding="ISO-8859-1")

In [55]:
df.head()

,date_time,avg_temp,precip_water,snow,snow_ice_depth
0,1948-09-01,71.5,0.00,0.0,0.0
1,1948-09-02,69.0,0.00,0.0,0.0
2,1948-09-03,72.0,0.00,0.0,0.0
3,1948-09-04,69.0,0.29,0.0,0.0
4,1948-09-05,72.0,0.06,0.0,0.0


In [56]:
score_df = df.loc[:, ["date_time", "avg_temp", "precip_water", "snow", "snow_ice_depth"]]

In [57]:
# df.insert(7, 'score', value)
score_df.head()

,date_time,avg_temp,precip_water,snow,snow_ice_depth
0,1948-09-01,71.5,0.00,0.0,0.0
1,1948-09-02,69.0,0.00,0.0,0.0
2,1948-09-03,72.0,0.00,0.0,0.0
3,1948-09-04,69.0,0.29,0.0,0.0
4,1948-09-05,72.0,0.06,0.0,0.0


In [58]:
for index, row in score_df.head(5).iterrows():
    print(row['avg_temp'])

71.5
69.0
72.0
69.0
72.0


In [59]:
score_df["precip"] = score_df["snow"] + score_df["snow_ice_depth"] + score_df["precip_water"]
   

In [60]:
score_df.head(5)


,date_time,avg_temp,precip_water,snow,snow_ice_depth,precip
0,1948-09-01,71.5,0.00,0.0,0.0,0.00
1,1948-09-02,69.0,0.00,0.0,0.0,0.00
2,1948-09-03,72.0,0.00,0.0,0.0,0.00
3,1948-09-04,69.0,0.29,0.0,0.0,0.29
4,1948-09-05,72.0,0.06,0.0,0.0,0.06


In [61]:
conditions = [
    (score_df['precip'] == 0.00) & (score_df['avg_temp'] >= 70) & (score_df['avg_temp'] <= 80)]
choices = [1]
score_df['score'] = np.select(conditions, choices, default=0)

In [62]:
score_df.head(5)


,date_time,avg_temp,precip_water,snow,snow_ice_depth,precip,score
0,1948-09-01,71.5,0.00,0.0,0.0,0.00,1
1,1948-09-02,69.0,0.00,0.0,0.0,0.00,0
2,1948-09-03,72.0,0.00,0.0,0.0,0.00,1
3,1948-09-04,69.0,0.29,0.0,0.0,0.29,0
4,1948-09-05,72.0,0.06,0.0,0.0,0.06,0


In [63]:
score_df.head()

,date_time,avg_temp,precip_water,snow,snow_ice_depth,precip,score
0,1948-09-01,71.5,0.00,0.0,0.0,0.00,1
1,1948-09-02,69.0,0.00,0.0,0.0,0.00,0
2,1948-09-03,72.0,0.00,0.0,0.0,0.00,1
3,1948-09-04,69.0,0.29,0.0,0.0,0.29,0
4,1948-09-05,72.0,0.06,0.0,0.0,0.06,0


In [64]:
score_df.to_csv('scored.csv')